# Índice (tentativo)

- [Introducción](#Introducción)
    - [Objetivos del cuaderno](#Objetivos-del-cuaderno)
    - [Motivación y marco teórico](#Motivación-y-Marco-Teórico)
        - [Dificultad clásica para resolver el problema de búsqueda de período]()
        - [HSP: Problema del subgrupo oculto]()
        - [Búsqueda de Período como instancia del HSP]()
        - [Preguntas exploratorias para la implementación del algoritmo de búsqueda de período]()
- [Diseño de Implementación]()

      


# Introducción

## Objetivos del cuaderno
 - Motivar el desarrollo de la implementación del algoritmo para la solución del problema de búsqueda de período.
 - Entender el problema que resuelve el algoritmo de búsqueda de período cuántico. Cuáles son sus componentes y sus marco teórico.
 - Presentar preguntas exploratorias 

## Motivación y Marco Teórico

La búsqueda del período $r$ de una función periódica $f$ consiste en encontrar el menor entero positivo $r$ tal que:

$$
f(x) = f(x + r) \quad \text{para todo } x \text{ en el dominio.}
$$

Generalmente, consideramos funciones definidas sobre un dominio grande, por ejemplo, un subconjunto de los enteros o números modulares, y con un codominio también amplio.


### ¿Por qué es difícil encontrar el período clásicamente?



Desde un punto de vista clásico, no existe un algoritmo eficiente conocido para encontrar $r$ sin evaluar la función en muchos puntos. Una estrategia natural es:

- Evaluar $f$ en distintos valores $x_1, x_2, \dots, x_n$.
- Buscar colisiones, es decir, pares $(x_i, x_j)$ con $i \neq j$ tales que:
$$f(x_i) = f(x_j)$$

Estas colisiones pueden indicar que la diferencia $|x_i - x_j|$ es un múltiplo del período $r$, lo que ayuda a deducir $r$.

**Número de comparaciones y probabilidad de colisión**

Con $n$ evaluaciones, el número de pares que podemos comparar es:

$$
\binom{n}{2} = \frac{n(n-1)}{2}.
$$

Si suponemos que $f$ toma valores en un conjunto de tamaño $r$ (por ejemplo, $\{0,1,\dots,r-1\}$), la probabilidad de que dos valores evaluados coincidan (colisión) aumenta con $n$.

Este fenómeno es análogo a la *paradoja del cumpleaños*: para que la probabilidad de encontrar al menos una colisión sea significativa (digamos, alrededor del 50%), se necesita aproximadamente:

$$
n \approx 1.2 \sqrt{r}
$$

evaluaciones.

**Complejidad temporal clásica**

En problemas relevantes, como aquellos relacionados con funciones definidas sobre entradas de $n$ bits, el período puede ser tan grande como $2^n$. Esto implica que la búsqueda clásica del período, basada en encontrar colisiones, requiere un número de evaluaciones y comparaciones del orden de:

$$
\mathcal{O}(\sqrt{2^n}) = \mathcal{O}(2^{n/2}),
$$

es decir, tiempo exponencial en $n$.

Esta dificultad es la base para la seguridad de ciertos sistemas criptográficos y motiva la búsqueda de algoritmos más eficientes, como los algoritmos cuánticos.



### Marco Teórico: Problema del Subgrupo Oculto (HSP)

El problema del subgrupo oculto (HSP, por sus siglas en inglés, *Hidden Subgroup Problem*) es un **marco matemático general** que subyace a varios de los algoritmos cuánticos más importantes que demuestran una **ventaja de velocidad exponencial** sobre los mejores algoritmos clásicos conocidos.

#### Fundamentos de Teoría de Grupos para Entender el HSP

Para comprender el **Problema del Subgrupo Oculto (HSP)**, necesitamos familiarizarnos con los siguientes conceptos clave de teoría de grupos:

---
- Grupo $(G, \cdot)$
Un **grupo** es un conjunto $G$ junto con una operación binaria $\cdot$ (como suma o multiplicación), que satisface cuatro propiedades:

1. **Clausura**: $\forall a, b \in G$, se tiene $a \cdot b \in G$.
2. **Asociatividad**: $(a \cdot b) \cdot c = a \cdot (b \cdot c)$.
3. **Elemento neutro**: Existe $e \in G$ tal que $a \cdot e = a$ para todo $a \in G$.
4. **Inversos**: Para cada $a \in G$, existe $a^{-1} \in G$ tal que $a \cdot a^{-1} = e$.

Además, se dice **abeliano** si la operación binaria que define la estructura del grupo es **conmutativa**. En otras palabras, el orden en que se realizan las operaciones con dos elementos no afecta el resultado. 

📌 **Ejemplo**: $(\mathbb{Z}, +)$ es un grupo, donde la operación es la suma. Además es abeliano.

- Subgrupo $K \leq G$
Un **subgrupo** $K$ es un subconjunto de $G$ que forma un grupo bajo la misma operación de $G$.

📌 **Ejemplo**: $2\mathbb{Z} \subseteq \mathbb{Z}$ es un subgrupo de los enteros: suma de pares sigue siendo par, el neutro es $0$, y cada número par tiene inverso aditivo.

- Cosets (Clases laterales)
Dado un subgrupo $K \leq G$ y un elemento $g \in G$, se define la **clase lateral izquierda** como:

$$
gK := \{g \cdot k : k \in K\}
$$

Estas clases particionan el grupo $G$, es decir, **cada elemento de $G$ pertenece a una única clase lateral**.  

📌 **Ejemplo**: En $G = \mathbb{Z}$ y $K = 3\mathbb{Z}$, las clases laterales son:
- $0 + 3\mathbb{Z} = \{\dots, -6, -3, 0, 3, 6, \dots\}$
- $1 + 3\mathbb{Z} = \{\dots, -5, -2, 1, 4, 7, \dots\}$
- $2 + 3\mathbb{Z} = \{\dots, -4, -1, 2, 5, 8, \dots\}$

- Función constante en clases laterales
Una función $f: G \to X$ es **constante en las clases laterales de $K$** si:

$$
f(g_1) = f(g_2) \quad \text{si y sólo si} \quad g_1K = g_2K
$$

📌 Es decir, **$f$ asigna el mismo valor a todos los elementos de una misma clase lateral, y valores distintos entre diferentes clases**.

En el **Problema del Subgrupo Oculto**, **no conocemos el subgrupo $K$**, pero sabemos que **la función $f$ está estructurada de forma que refleja las clases laterales de $K$**.

---

#### Definición del Problema del Subgrupo Oculto (HSP)

El problema del subgrupo oculto se define formalmente como sigue:

> Dada una **función $f$ de un grupo $G$ generado finitamente a un conjunto finito $X$**, con la propiedad de que **$f$ es constante en las cosets de un subgrupo $K$ y distinta en cada coset**. Es decir,  
> 
> $$f(g_1) = f(g_2) \quad \Leftrightarrow \quad g_1 K = g_2 K.$$
> 
> 
> Se tiene acceso a una "caja negra cuántica" que realiza la transformación unitaria  
> 
> $$U_f |g⟩|h⟩ = |g⟩|h \oplus f(g)⟩,$$
> 
> donde $g \in G$, $h \in X$, y $\oplus$ es una operación binaria en $X$.  
> 
> El objetivo es encontrar un conjunto generador del subgrupo oculto $K$.

Ejemplos notables de problemas que son instancias del HSP:
- Factorización de números enteros (algoritmo de Shor)
- Logaritmo discreto
- Búsqueda de órdenes
- Búsqueda de períodos
- Algoritmo de Deutsch-Jozsa
- Problema de Simon

---

#### Ventaja Exponencial para Grupos Abelianos Finitos

Los algoritmos cuánticos que resuelven el HSP para **grupos abelianos finitos** exhiben una clara **ventaja exponencial** en eficiencia computacional gracias a la implementación eficiente de la **Transformada de Fourier Cuántica (QFT)**:

- Para $G$ abeliano y finito, la QFT se puede realizar en **tiempo polinomial en $\log |G|$**.
- Por ejemplo, el algoritmo de Shor para factorización (una instancia del HSP sobre $\mathbb{Z}_N$) tiene complejidad cuántica de  
  $$
  O(n^2 \log n \log \log n),
  $$
  mientras que el mejor algoritmo clásico conocido (el *number field sieve*) tiene complejidad  
  $$
  \exp(\Theta(n^{1/3} \log^{2/3} n)).
  $$
- Esto representa una **aceleración exponencial real**.

---

#### El *Promise Problem*

El HSP es un **problema con promesa** (*promise problem*). Es decir, se **supone garantizado** que la función de entrada $f$ cumple la propiedad de ser constante en las clases laterales de un subgrupo $K$.

- Esta estructura es explotada por el algoritmo cuántico para obtener una solución eficiente.
- Sin esta promesa (es decir, si $f$ es arbitraria), **no se puede garantizar una ventaja cuántica exponencial**.

Esto ilustra que las **ventajas cuánticas no son universales**, sino que aparecen en problemas con estructuras algebraicas aprovechables.

---

#### Principios de Mecánica Cuántica Aprovechados

Los algoritmos para el HSP se basan principalmente en dos principios de la mecánica cuántica:

1. Superposición (*Paralelismo Cuántico*)
- Un estado como  
  $$
  |0⟩^{\otimes n}|0⟩ \xrightarrow{\text{Hadamard}} \frac{1}{\sqrt{2^n}} \sum_{x \in \{0,1\}^n} |x⟩|0⟩
  $$
  permite evaluar $f(x)$ **simultáneamente** para todos los $x$ aplicando la caja negra (oráculo):
  $$
  \frac{1}{\sqrt{2^n}} \sum_{x} |x⟩|f(x)⟩.
  $$

2. Interferencia
- Después de aplicar $U_f$, la información relevante sobre $K$ no está directamente observable.
- Se aplica la **Transformada de Fourier Cuántica (QFT)** sobre el primer registro.
- La QFT permite que ciertas amplitudes se **interfieran constructiva o destructivamente**, revelando la estructura del subgrupo oculto.

---

#### Esquema General del Algoritmo Cuántico para el HSP

1. Preparación del estado inicial
- Dos registros:
  - El primero se pone en superposición uniforme:  
    $$
    \frac{1}{\sqrt{|G|}} \sum_{g \in G} |g⟩
    $$
  - El segundo se inicializa como $|0⟩$ o en una superposición conveniente.

2. Aplicación de la caja negra
- Se aplica $U_f$:
  $$
  \frac{1}{\sqrt{|G|}} \sum_{g \in G} |g⟩|f(g)⟩.
  $$

3. Aplicación de la QFT
- Se aplica la **QFT inversa** al primer registro.
- Esto transforma la información de periodicidad en **picos de amplitud** que codifican información sobre $K$.

4. Medición
- Se mide el primer registro.
- El resultado está relacionado con el **espacio dual** del grupo, ortogonal a $K$.

5. Post-procesamiento clásico
- A partir de múltiples mediciones, se aplica un algoritmo clásico (como el de **fracciones continuas**) para **reconstruir los generadores del subgrupo $K$**.


### Búsqueda de período

El problema de búsqueda de período es un caso especial del HSP. En este contexto, la búsqueda de período se puede ver como la identificación de un subgrupo oculto en un grupo abeliano finito.

El problema se define de la siguiente manera: 

> Suponga que tiene una función $f$ que produce una **salida de un solo bit** y es **periódica**, de tal forma que $f(x + r) = f(x)$ para un período desconocido $r$ (donde $0 < r < N=2^L$).
> Se le proporciona una "caja negra cuántica" (u oráculo) $U_f$ que realiza la transformación unitaria $U_f|x⟩|y⟩ → |x⟩|y \oplus f(x)⟩$ (donde $\oplus$ denota la suma módulo 2). El objetivo es determinar el valor de $r$.

Esta descripción encaja dentro del marco del HSP si se eligen:

- $G= \mathbb{Z}_N$ como grupo abeliano finito,

- $K= \langle r \rangle$ como subgrupo generado por el período $r$,

- $f$ constante en cosets $x+K$, y distinta en cosets diferentes.

Para que esta formulación sea una instancia estricta del HSP sobre $\mathbb{Z}_N$, se requiere que el período $r$ divida a $N$, es decir, $r\mid N$.

Si $r \nmid N$, la función puede seguir siendo periódica en sentido práctico, pero no define una partición del grupo en cosets de un subgrupo válido, y por lo tanto no es formalmente una instancia del HSP sobre $\mathbb{Z}_N$. Sin embargo, el algoritmo cuántico sigue funcionando si se elige $N \gg r$, como sucede en el algoritmo de Shor, porque cuando se aplica la Transformada de Fourier Cuántica (QFT) sobre el grupo $\mathbb{Z}_N$​ a un estado que codifica una función periódica de período $r$, el resultado tiene máxima amplitud (picos) en ciertos estados $m\in \mathbb{Z}_N$​, tales que: $\frac{m}{N} \approx \frac{s}{r}$, donde $s\in Z$ es un entero desconocido entre $0$ y $r−1$. 

### Preguntas Exploratorias 

#### Robustez estructural del algoritmo

1. ¿Qué pasa si el período $r$ es impar o no potencia de 2?

- Relevancia: ¿la estructura de $r$ afecta la fidelidad de la recuperación?
- Implementación: comparar varios valores de $r$: $r=4,5,6,7,8$ todos con mismo $N$.
- Hipótesis: no ser potencia de 2 no impide la recuperación, pero puede generar distribuciones de salida más dispersas.

2. ¿Qué pasa si la función no distingue bien los cosets?

- Relevancia: viola la hipótesis del HSP .
- Implementación: construir una función periódica que repite el mismo valor en algunos cosets distintos.
- Hipótesis: la interferencia cuántica se degrada, disminuye la probabilidad de picos relevantes.

#### Robustez frente a ruido

3. ¿Qué ocurre si se corrompe la función $f$ en un subconjunto del dominio?

- Relevancia: simula errores en el oráculo.
- Implementación: alterar aleatoriamente algunos valores de $f$.
- Hipótesis: el algoritmo tolera corrupción leve (estudiar porcentaje), pero errores en múltiples cosets destruyen la señal.

#### Variantes
4. ¿Qué ocurre si la función tiene más de un período (función multiperiódica)?

- Relevancia: generalización interesante.
- Implementación: definir una función como $f(x)=g(x mod r_1)+h(x mod r_2)$.
- Hipótesis: la QFT muestra varios picos y se puede identificar un mínimo común múltiplo, aunque el algoritmo estándar no lo capture directamente.

5. ¿Qué hace el algoritmo con una función $f(x)=x+g(x)$ con g una función periódica?

- Relevancia: Esta función no es periódica, por lo tanto viola la promesa del HSP. Sirve como contraejemplo para evaluar qué pasa cuando la función no tiene simetría de cosets.
- Implementación: Definir $g(x)=(−1)^{x mod r}+1$ u otra periódica. Aplicar el algoritmo de búsqueda de período con esta $f(x)$.
- Hipótesis: La QFT no debería producir picos definidos. Se espera una distribución difusa o aleatoria y fracaso del algoritmo al intentar recuperar un período.

- ¿El algoritmo encuentra algún falso positivo?
- ¿Qué distribución produce la QFT?
- ¿Cómo varía al cambiar el período de $g$ o al eliminar la parte lineal $x$?

6. ¿Qué hace el algoritmo con una función $f(x)=x \mod r +g(x)$ con g una función periódica?
- Relevancia: Esta función sí puede ser periódica si $g$ tiene el mismo período $r$. Es un caso no trivial que puede seguir cumpliendo con la estructura del HSP.
- Implementación: Tomar $g(x)$ de período $r$ o $r′$, comparar ambos casos. Aplicar el algoritmo cuántico y observar la distribución tras la QFT.
- Hipótesis:
    - Si $per⁡(g)=r$, entonces $f$ es periódica $\implies$ se espera recuperación correcta del período.
    - Si $per⁡(g)\neq r$, la periodicidad se altera $\implies$ la QFT podría mostrar picos espurios o desfasados.

# Diseño de Implementación

## Definición de la función

Para validar la robustez estructural del algoritmo, elegimos una función períodica basada en permutaciones cíclicas. 

$$f(x)=g^x(s_i)$$

donde:

- $s_i \in S_r = (s_1, s_2, \dots, s_r)$, un elemento de una cadena de longitud $r$.
- $g$ es una permutación cíclica que actúa sobre $S_r$ tal que:
    - $g(s_1)= s_2, \dots, g(s_{r-1})=s_r, g(s_r)=s_1$, y para cualquier $y\notin S_r, g(y)=y$
- $g^x$ es la composición iterada de $g$ consigo misma $x$ veces.

Esta función es periódica debido a que:

- Una permutación cíclica (o ciclo) de longitud $r$ es una permutación que mueve cíclicamente $r$ elementos y deja fijos los demás. Luego, por definición, $g$ es un $r$-ciclo.
- El orden (periodo) de la permutación $g$ es $r$, es decir, $g^r=id$ (la identidad).
    - Si $y \neq S_r$,$g(y)=y \implies g^r(y)=y \implies g^r = id$
    - Si $y \in S_r \implies \exists i \text{ tal que } y=s_i \implies g(s_i)= \begin{cases} s_{i+1} & \text{ si } i\neq r \\ s_1 & \text{ si }i=r \end{cases}$ $ \implies g^r(s_i) = s_{i+r \mod r}=s_i \implies g^r=id$

Finalmente, $f(x+r)= g^{x+r}(s_i)= g^x(g^r(s_i))= g^x(s_i) = f(x)$, es decir $f$ es periódica con período $r$.

Esta elección nos permite explorar la robustez del algoritmo debido a que el dominio de entrada puede ser mayor que $r$ y no necesariamente múltiplo de $r$. Está diseñada de modo tal que dos entradas diferentes caen en la misma clase de equivalencia, pero de manera no evidente, lo cual permite estudiar la precisión del algoritmo en distinguir estados cercanos.

El uso de esta función permite abordar preguntas exploratorias relevantes, tales como:

- ¿El algoritmo encuentra falsos positivos cuando el período es impar o poco visible?

- ¿Qué tipo de distribución produce la QFT aplicada al estado cuántico resultante?

- ¿Cómo se ve afectado el rendimiento del algoritmo al aumentar la complejidad estructural de la función?

Este tipo de análisis no sería posible con funciones triviales, como $f(x)= x \mod r$ o funciones constantes, que ofrecen poca riqueza espectral.




## Oráculo

La implementación de oráculos cuánticos se basa en construir un operador unitario reversible que codifica la función $f$.

Para implementar el oráculo cuántico que evalúa la función $f(x)=g^x(s_i)$, debemos construir un circuito que, dado un estado $\ket{x}$, transforme un registro auxiliar $\ket{0}$ en $\ket{f(x)}$, es decir: 
$$\ket{x}\ket{0} \overset{U_f}{\to} \ket{x}\ket{f(x)}= \ket{x}\ket{g^x(s_i)}$$

- Para el **registro de entrada** tendremos $n$ qubits que codifican el valor de $x \in \{0, \dots, 2^n-1\}$.
- Para el **registro de salida** tendremos $n$ qubits también, que codifican el valor de $f(x) \in \{0, \dots, 2^n-1\}$.

Para ello podemos implementar clásicamente $g$ y luego aplicarlo iterativamente $x$ veces. Finalmente implementamos $U_f$ para almacenar ese resultado en el registro de salida.

In [19]:
def generar_permutacion_ciclica(S_r, N):
    """
    Implementación clásica de g sobre todo el dominio.
    
    Genera una permutación cíclica que actúa como ciclo sobre los elementos de S_r,
    y fija los restantes elementos del dominio {0, ..., N-1}.
    """
    permutacion = {s: s for s in range(N)}  # Inicialmente identidad
    r = len(S_r)
    for i in range(r):
        permutacion[S_r[i]] = S_r[(i + 1) % r]
    return permutacion

def aplicar_g_iterado(s, permutacion, x):
    """
    Implementación clásica de f sobre un elemento del dominio.
    
    Aplica la permutación 'permutacion' x veces sobre el valor s.
    """
    for _ in range(x):
        s = permutacion[s]
    return s



Para testear esta función tomamos un período interesante para explorar pero no tan grande para no tener un registro de entrada muy grande y aún así obtener una buena probabilidad de recuperación de r.

In [21]:
import random #Para generar S_r

# TEST 1 de la implementación clásica de f
n = 6 # Así obtenemos una probabilidad de recuperación de r alta
r = 7 #Tomamos un período interesante, no par

N = 2**n #Tamaño del registro de entrada y salida.

S_r = random.sample(range(0, N-1), r)
s = random.choice(S_r) #Elegimos el elemento inicial para aplicar g

Y = generar_permutacion_ciclica(S_r, N) # Imagen de g sobre el registro de entrada 
for x in range(0, N-1):
    print(f"f({x}) = g^{x}({s}) = {aplicar_g_iterado(s, Y, x)}")

f(0) = g^0(7) = 7
f(1) = g^1(7) = 15
f(2) = g^2(7) = 52
f(3) = g^3(7) = 3
f(4) = g^4(7) = 54
f(5) = g^5(7) = 34
f(6) = g^6(7) = 62
f(7) = g^7(7) = 7
f(8) = g^8(7) = 15
f(9) = g^9(7) = 52
f(10) = g^10(7) = 3
f(11) = g^11(7) = 54
f(12) = g^12(7) = 34
f(13) = g^13(7) = 62
f(14) = g^14(7) = 7
f(15) = g^15(7) = 15
f(16) = g^16(7) = 52
f(17) = g^17(7) = 3
f(18) = g^18(7) = 54
f(19) = g^19(7) = 34
f(20) = g^20(7) = 62
f(21) = g^21(7) = 7
f(22) = g^22(7) = 15
f(23) = g^23(7) = 52
f(24) = g^24(7) = 3
f(25) = g^25(7) = 54
f(26) = g^26(7) = 34
f(27) = g^27(7) = 62
f(28) = g^28(7) = 7
f(29) = g^29(7) = 15
f(30) = g^30(7) = 52
f(31) = g^31(7) = 3
f(32) = g^32(7) = 54
f(33) = g^33(7) = 34
f(34) = g^34(7) = 62
f(35) = g^35(7) = 7
f(36) = g^36(7) = 15
f(37) = g^37(7) = 52
f(38) = g^38(7) = 3
f(39) = g^39(7) = 54
f(40) = g^40(7) = 34
f(41) = g^41(7) = 62
f(42) = g^42(7) = 7
f(43) = g^43(7) = 15
f(44) = g^44(7) = 52
f(45) = g^45(7) = 3
f(46) = g^46(7) = 54
f(47) = g^47(7) = 34
f(48) = g^48(7) = 62
f(49)

### Estrategia de implementación de $U_f$

Tenemos dos registros de $n$ qubits.

- Registro de entrada $\ket{x}$
- Registro de salida $\ket{0}$

La base canónica (computacional) está dada por $\{ \ket{x}\ket{y} \}_{x,y \in \{0,1\}^n} $. El operador $U_f$, de tamaño $2^{2n}\times 2^{2n}$ actúa como:

$$U_f\ket{x}\ket{y}= \ket{x}\ket{y \oplus f(x)} \iff U_f = \sum_{x,y} \ket{x} \bra{x} \otimes \ket{y \oplus f(x)} \bra{y}$$


En particular, si $y=0$: 
$$\ket{x}\ket{0} \overset{U_f}{\to} \ket{x}\ket{f(x)}= \ket{x}\ket{g^x(s_i)}$$



#### ¿Cómo podemos definir U_f de manera algebraica?

Primero veamos cómo representar $U_g: \{\ket{0}, \dots, \ket{N-1}\} \to \{\ket{0}, \dots, \ket{N-1}\}$ tal que $U_g\ket{y}= \ket{g(y)}$. 

Dado que $g$ aplica sobre el dominio una permutación, se puede representar, de manera general, mediante la matriz de permutación $U_g=P = (P_{ij}), \text{ con } P_{ij}= \begin{cases} 1 & \text{ si }i=g(j) \\ 0 & \text{ si no} \end{cases}$
   
   Porque la columna $y$ de $P$ es el vector $\ket{g(y)}$.
    $$U_g= \sum_{y=0}^{N-1} \ket{g(y)}\bra{y}= \sum_{y \in S_r} \ket{g(y)}\bra{y} +  \sum_{y \notin S_r} \ket{y}\bra{y}\implies U_g\ket{y} = \ket{g(y)}\bra{y}\ket{y}= \ket{g(y)}$$

Dado que $f(x)=g^x(s)$, queremos la acción de $g$ aplicada $x$ veces. 

$$U_g^2=U_gU_g = \sum_y \ket{g(y)}\bra{y} \sum_{z}\ket{g(z)}\bra{z}= \sum_{y,z} \ket{g(y)}\bra{y}\ket{g(z)}\bra{z}$$

Como $\bra{y}\ket{g(z)}= \begin{cases}0 & \text{ si } y\neq g(z) \\ 1 & \text{ si } y= g(z)\end{cases}$, sobreviven los términos con $y=g(z)$. Colapsamos esta suma doble imponiendo esta condición. Luego, dado que $g$ es biyectiva podemos renombrar $z \to g(z)=y$.

$$U_g^2 = \sum_{z} \ket{g(g(z))}\bra{z} = \sum_z \ket{g^2(z)}\bra{z}$$

Finalmente, por inducción se obtiene 
$$U_g^x= (U_g)^x = \sum_z \ket{g^x(z)}\bra{z}$$

Busquemos construir $U_f$ a partir de $U_g^x$.

$$U_f: \ket{x}\ket{y} \to \ket{x}\ket{y \oplus g^x(s)}$$

De manera similar a la definición de $U_g$, tenemos que:

$$U_f = \sum_{x',y'} \ket{x'}\ket{y' \oplus g^{x'}(s)}\bra{x'}\bra{y'} \implies U_f \ket{x}\ket{y} = \ket{x}\ket{y \oplus g^{x}(s)}\bra{x}\bra{y} \ket{x}\ket{y} = \ket{x}\ket{y \oplus g^{x}(s)}$$



#### Descomposición de $U_f$ en compuertas elementales

Consideramos el desarrollo binario de $x= \sum_{k=0}^{n-1}x_k2^k$:

$$g^x(s)= g^{2^0x_0} \circ g^{2^1x_1} \circ \dots \circ g^{2^{n-1}x_{n-1}}(s) $$

Podemos implementar esto con $n$ bloques controlados. Cada bloque aplica el operador $U_{g^{2^k}}$ de manera controlada por el qubit $x_k$ del registro de entrada.

Supongamos que queremos aplicar la permutación $g^{2^k}$ sobre un subconjunto $S_r$ del dominio de entrada. Cada permutación puede representarse como una lista de transiciones $(s_i, s_j)$, es decir, que $g^{2^k}(s_i) = s_j$.

La idea es que cada par $(s_i, s_j)$ indica que si el registro de salida está en el estado $\ket{s_i}$, debe transformarse a $\ket{s_j}$. Quiza podemos implementar esto simulando un mapeo mediante compuertas cuánticas controladas.

Podemos simular el comportamiento de un multiplexor: según el estado de entrada, aplica distintas salidas. Pero en lugar de hacerlo con un gran bloque, se descompone en compuertas elementales, que pueden implementarse en cualquier arquitectura cuántica universal. Esto para aplicar cada $U_{g^{2^k}}$ de manera controlada por el qubit $x_k$, como tenemos a los sumo $k=n-1$, será lineal.



#### Simulación

Para verificar la correcta implementación, simulamos el oráculo en un caso concreto.